In [4]:
import time
import pyvisa as visa
import datetime
import os
import sys
import codecs
import codecs
import pandas as pd
import serial
import numpy as np
from shutterapi import *
import monochromatorapi as mcapi # Import monochromator api
import clr # Import the .NET class library

In [5]:
def picoammeter_initialize(Ch1ON=1,Ch2ON=1,interval=0.1,nsamples=50,asrl="asrl5::instr",debug=False):
        def KISend(command):
                picoa.write(command)
                if debug:
                        print('Sent '+ f'{command}')
        def KIRequest(command):
                response = picoa.query(command)
                if debug:
                        print( 'Sent '+command)
                        print( 'Received '+response.strip())
                return(response)


        Ch1ON = Ch1ON  # 0 = channel off, 1 = channel on
        Ch2ON = Ch2ON  # 0 = channel off, 1 = channel on
        asrl=asrl  # asrl = "asrl5::instr" #instrument address for picoammeter
        runtime = interval * nsamples
        print(f"samples will be taken in a {runtime} second exposure")
        """Channel 1"""
        Ch1ON = Ch1ON
        Ch1ULimit = .01  # current upper auto range limit 
        Ch1LLimit = 1e-7  # current lower auto range limit
        Ch1NPLC = 10.0 # Measurement Speed in NPLC, 
        """ Note on Ch NPLC 
        Options: 0.01 for fast readout mode,FAST: Sets speed to 0.01 PLC and sets display resolution to 3½ digits.
        • MED: Sets speed to 0.10 PLC and sets display resolution to 4½ digits.
        • NORMAL: Sets speed to 1.00 PLC and sets display resolution to 5½ digits.
        • HI ACCURACY: Sets speed to 10.00 PLC and sets display resolution to 6½ digits.
        • OTHER: Use to set speed to any PLC value from 0.01 to 10. Display resolution is not
        changed when speed is set with this option."""
        Ch1SrcV = 0  # Source Voltage
        """Channel 2"""
        Ch2ON = Ch2ON
        Ch2ULimit = .01  # current upper auto range limit 
        Ch2LLimit = 1e-7  # current lower auto range limit 
        Ch2NPLC = 10.0 # Measurement Speed in NPLC
        Ch2SrcV = 0  # Source Voltage
        if Ch1ON + Ch2ON < 1:
                print( "This program requires at least one measurement channel:  "+str(Ch1ON+Ch2ON)+" selected" )
                return
        try:
                rm = visa.ResourceManager() #create PyVISA instrument session with 6482
                picoa = rm.open_resource(asrl) #asrl# could change depending on the serial port used COM3 goes to asrl3::instr
                picoa.write_termination='\r'
                picoa.read_termination='\r'
                picoa.baud_rate = 9600
                picoa.data_bits = 8
                picoa.parity = visa.constants.Parity.none
                picoa.flow_control = visa.constants.VI_ASRL_FLOW_NONE
                # csvpath = os.getcwd( )+'\\' #we should change this to a user defined path. 
                '''
                Set up 6482 communications from the 6482 front panel: RS-232, 9600 Baud, 8 data bits, No parity, No Flow Control, CR terminator
                USE < and > Edit keys and Enter key to select values.
                Menu -> Communication -> RS-232 (if not already RS-232, you'll hear the instrument click, then repeat above)
                                        -> BAUD -> 9600
                                        -> BITS -> 8
                                        -> PARITY -> NONE
                                        -> TERMINATOR -> <CR>
                                        -> FLOW-CTRL -> NONE
                '''
                # other globals
                outputqueue=[]
                debug=1

                
                outputqueue.append(KIRequest('*IDN?')+'\n')
                print( outputqueue[0]+'added to output queue')
                KISend('*RST') #reset instrument
                #wait for instrument to complete reset
                val = 0
                while val != 1:
                        val = int( KIRequest('*OPC?'))
                # set channel parameters, data format, etc.
                if Ch1ON:
                        KISend(':SENS1:CURR:RANG:AUTO ON')
                        KISend(':SENS1:CURR:RANG:AUTO:ULIM '+str(Ch1ULimit))
                        KISend(':SENS1:CURR:RANG:AUTO:LLIM '+str(Ch1LLimit))
                        KISend(':SENS1:CURR:NPLC '+str(Ch1NPLC))
                        KISend(':SOUR1:VOLT:RANGE:AUTO 1 ')
                        KISend(':SOUR1:VOLT '+str(Ch1SrcV))
                        KISend('OUTP1 ON')
                if Ch2ON:
                        KISend(':SENS2:CURR:RANG:AUTO ON')
                        KISend(':SENS2:CURR:RANG:AUTO:ULIM '+str(Ch2ULimit))
                        KISend(':SENS2:CURR:RANG:AUTO:LLIM '+str(Ch2LLimit))
                        KISend(':SENS2:CURR:NPLC '+str(Ch2NPLC))
                        KISend(':SOUR2:VOLT:RANGE:AUTO 1 ')
                        KISend(':SOUR2:VOLT '+str(Ch2SrcV))
                        KISend('OUTP2 ON')
                # formatelements = ''
                # if Ch1ON:
                #         formatelements = formatelements+'CURR1'
                # if Ch1ON and Ch2ON:
                #         formatelements = formatelements+','
                # if Ch2ON:
                #         formatelements = formatelements+'CURR2'
                # KISend(':FORM:ELEM '+formatelements)
        except Exception as ex:
                msg =f"Could not read using picoammeter. Error: {ex}"
                print(msg)
                return
        return(picoa)

def picoammeter_end(picoa): 
        try: 
                picoa.close()
                print("Measurements complete")
        except: 
                print("Error! Device already closed or incorrect device specified")

def PICOA_Send(picoa,command,debug=False):
        picoa.write(command)
        if debug:
                print( 'Sent '+f'{command}')
def PICOA_Request(picoa,command,debug=False):
        response = picoa.query( command )
        if debug:
                print( 'Sent '+command)
                print( 'Received '+response.strip())
        return(response)
def picoa_get_measurement(picoa,filename,interval=0.1,nsamples=50): 
    interval = interval #time (s) between consecutive writes of selected channel readings to datalog
    nsamples = nsamples  # number of readings total written to datalog
    count=1
    filename=filename
    StartTime = time.time()
                # SampTime = 0
                # for i in range(1 ,nsamples):
                #         # mark this measurement's time
                #         MeasTime = (i-1) * interval
                #         SampTime = time.time()-StartTime
                #         # Make a measurement, append MeasTime to outputqueue line
                #         MeasLine = KIRequest(':READ?').strip()+', '+str(SampTime)
                #         if debug:
                #                 print( MeasLine+'added to output queue')
                #         outputqueue.append(MeasLine+'\n')
    while count<=nsamples:
        if count==1:  
            rawout=PICOA_Request(picoa,':READ?').strip()+','+f'{time.time()-StartTime}'
            outlist=[[float(s) for s in rawout.split(',')]]
            # outdf=pd.DataFrame(outlist)
            # outdf.columns=['Ch1','Ch2']
            # print(count)
        else: 
            rawout=PICOA_Request(picoa,':READ?').strip()+','+f'{time.time()-StartTime}'
            outlist.append([float(s) for s in rawout.split(',')])
            # outdf.append(pd.DataFrame(outlist, columns=['Ch1','Ch2']),ignore_index=True)
            # print(count)
        count+=1
    outdf=pd.DataFrame(outlist)
    outdf.columns=['Ch1','Ch2','Elapsed_time']
    outdf.to_csv(filename)
    return outdf

In [10]:
def picoa_set_folder(exp_folder,parent_diretory):
    """
    Create new folder to store the experiment files and subfiles.
        Inputs:
            :exp_folder(string): folder name
            :parent_directory(string): location of folder
        Returns:
            ::folder and directory creation messages
            ::error message, if error code is 17, updates folder path and directory
    """
    folder_location=parent_diretory+'\\'+exp_folder
    import os # importing os module   
    path = folder_location # path
    # Create the directory in '/home / User / Documents' 
    try: 
        os.mkdir(path)
        customdirectory=path
        print(f"Created empty folder {customdirectory} for storing experiment files")
    except OSError as error: 
        out=error
        if out.errno==17: 
            customdirectory=path
            print(error)
            print(f"Path updated to existing {customdirectory}. Check that the folder is empty before proceeding.")
            return(customdirectory)

In [11]:
#Scan infomration 
start_wl=float(115.0) #nm
end_wl=float(700.0)#nm
wl_step=float(1) #nm 
slitsize= 1000#micron
"""Remember to physically switch the lamp on the monochromator"""
Lamp='D2' #lamp switched to d2 on 10 AM 7/14/2022
filternum=1 
#File_save settings. 
parent_directory=r'C:\Users\User\Documents\UV_Monochromator_Laptop\UV-Monochromator-control-2'
exp_filenames_basename=f'Photodiode_Flip_Exp3_{Lamp}_slit_{slitsize}micron_Filter_{filternum}'
exp_folder_name=exp_filenames_basename+f'_{int(start_wl)}nmto{int(end_wl)}nm'
exp_filenames_basename_dark=exp_filenames_basename+'_dark'
exp_directory=picoa_set_folder(exp_folder_name,parent_directory)

Created empty folder C:\Users\User\Documents\UV_Monochromator_Laptop\UV-Monochromator-control-2\Photodiode_Flip_Exp3_D2_slit_1000micron_Filter_1_115nmto700nm for storing experiment files


In [12]:
# MCPort = 'COM4'

# mcapi.go_to_from(MCPort,600,610) #power recycling issue give it a little kick of 10 nm 

In [ ]:
MCPort = 'COM4'
mcapi.home(MCPort)
mcapi.whereishome()

In [13]:
# Ch1ON=1 #Channel 1 ON
# Ch2ON=1 #Channel 2 ON 
# nsamples=30 #previously 100 
# interval=0.1 #no change. 
# #intiallize picoammeter with the settings. 
# asrl="asrl5::instr" #port for the picoammeter
# picoa=picoammeter_initialize(Ch1ON,Ch2ON,interval,nsamples,asrl,debug=False)
# shutterport= 'COM3'
# MCPort = 'COM4'
# #mcapi.home(MCPort)
# mcapi.go_to_fromhome(MCPort,start_wl)
# current_wl=start_wl
# #take some dark measurement
# shutclose(shutterport)
# dark_filename=exp_folder_name+'\\'+exp_filenames_basename_dark+'.csv'
# data=picoa_get_measurement(picoa,dark_filename,interval,nsamples)
# #Scanning between the start and the stop wavelengths 
# while current_wl<=end_wl:    
#     filename=exp_folder_name+'\\'+exp_filenames_basename+f'_wl_{current_wl}nm'+'.csv'
#     print(f"Taking data for {current_wl}")

#     shutopen(shutterport)
#     data=picoa_get_measurement(picoa,filename,interval,nsamples)
#     shutclose(shutterport)

#     print(f"Going to {current_wl+wl_step} nm")
#     mcapi.go_to_from(MCPort,current_wl,float(current_wl+wl_step))
#     current_wl=current_wl+wl_step
# picoammeter_end(picoa)
# print(f"All data taken and stored in {exp_directory}")
# print("Monochromator is going home!")
# mcapi.home(MCPort)

In [14]:
Ch1ON=1 #Channel 1 ON
Ch2ON=1 #Channel 2 ON 
nsamples=30 #previously 100 
interval=0.1 #no change. 
#intiallize picoammeter with the settings. 
asrl="asrl5::instr" #port for the picoammeter
picoa=picoammeter_initialize(Ch1ON,Ch2ON,interval,nsamples,asrl,debug=False)
shutterport= 'COM3'
MCPort = 'COM4'
#mcapi.home(MCPort)
mcapi.go_to_fromhome(MCPort,start_wl)
current_wl=start_wl
#take some dark measurement
shutclose(shutterport)
print("Taking pre dark")
dark_filename=exp_folder_name+'\\'+exp_filenames_basename_dark+'_pre.csv'
data=picoa_get_measurement(picoa,dark_filename,interval,nsamples)
counter =int(1)
#Scanning between the start and the stop wavelengths 
while current_wl<=end_wl:
    counter=counter+int(1) #counter for take dark for every 10 lamp exposures 
    filename=exp_folder_name+'\\'+exp_filenames_basename+f'_wl_{current_wl}nm'+'.csv'
    print(f"Taking data for {current_wl}")
    shutopen(shutterport)
    data=picoa_get_measurement(picoa,filename,interval,nsamples)
    shutclose(shutterport)
    if counter==int(10): #taking dark for every 10 lamp exposures 
        counter=int(1)
        dark_filename=exp_folder_name+'\\'+exp_filenames_basename_dark+f'_wl_{current_wl}nm'+'.csv'
        data=picoa_get_measurement(picoa,dark_filename,interval,nsamples)
        print("Taking wl dark")
    print(f"Going to {current_wl+wl_step} nm")
    mcapi.go_to_from(MCPort,current_wl,float(current_wl+wl_step))
    current_wl=current_wl+wl_step
print("Taking post dark")
dark_filename=exp_folder_name+'\\'+exp_filenames_basename_dark+'_post.csv'
data=picoa_get_measurement(picoa,dark_filename,interval,nsamples)
picoammeter_end(picoa)
print(f"All data taken and stored in {exp_directory}")
print("Monochromator is going home!")
mcapi.home(MCPort)


samples will be taken in a 3.0 second exposure
Sent *IDN?
Received KEITHLEY INSTRUMENTS INC.,MODEL 6482,4446862,A02   Jan 18 2017 10:20:01/A02  /F
KEITHLEY INSTRUMENTS INC.,MODEL 6482,4446862,A02   Jan 18 2017 10:20:01/A02  /F
added to output queue
Sent *RST
Sent *OPC?
Received 1
Sent :SENS1:CURR:RANG:AUTO ON
Sent :SENS1:CURR:RANG:AUTO:ULIM 0.01
Sent :SENS1:CURR:RANG:AUTO:LLIM 1e-07
Sent :SENS1:CURR:NPLC 10.0
Sent :SOUR1:VOLT:RANGE:AUTO 1 
Sent :SOUR1:VOLT 0
Sent OUTP1 ON
Sent :SENS2:CURR:RANG:AUTO ON
Sent :SENS2:CURR:RANG:AUTO:ULIM 0.01
Sent :SENS2:CURR:RANG:AUTO:LLIM 1e-07
Sent :SENS2:CURR:NPLC 10.0
Sent :SOUR2:VOLT:RANGE:AUTO 1 
Sent :SOUR2:VOLT 0
Sent OUTP2 ON
home circuit enabled, prepared to home
scanner is above home so moving down to home
decreasing wavelength at 23KHz
Limits status is:2
Limits status is:34
scan controller stopped
decreasing wavelength for 3 revolutions
increasing wavelength for 2 revolutions
high accuracy circuit enabled
finding edge of home flag at 4500KHz th